	
## Poultry Audio Classification with Deep Learning and Burn Layer Fusion

This notebook implements a deep learning-based approach for classifying poultry audio signals, inspired by the paper "Optimizing poultry audio signal classification with deep learning and burn layer fusion".

The model uses a custom Burn Layer to enhance robustness by injecting controlled random noise during training.



In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

import random
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Check if GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

/Users/farhanmashrur/Desktop/cds/avian_alert/tf-metal-env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Custom Burn Layer Implementation

The Burn Layer is a key innovation from the paper that adds controlled random noise during training to improve model robustness.

In [4]:
class BurnLayer(layers.Layer):
    def __init__(self, burn_intensity=0.2, **kwargs):
        super(BurnLayer, self).__init__(**kwargs)
        self.burn_intensity = burn_intensity
    
    def call(self, inputs, training=None):

        if training:

            return inputs + self.burn_intensity * tf.random.normal(shape=tf.shape(inputs))
        else:

            return inputs
    
    def get_config(self):
        config = super(BurnLayer, self).get_config()
        config.update({"burn_intensity": self.burn_intensity})
        return config

## Audio Feature Extraction and Preprocessing

In [5]:
def extract_features(audio_path, sr=44100, duration=2.0, n_mfcc=20):
    """
    Extract audio features from a file
    """
    try:
        # Load audio file with specified sample rate and duration
        y, sr = librosa.load(audio_path, sr=sr, duration=duration)
        
        # If audio is shorter than duration, pad it
        if len(y) < int(duration * sr):
            y = np.pad(y, (0, int(duration * sr) - len(y)))
        
        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        
        # Extract chromagram
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        
        # Extract spectral contrast
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        
        # Calculate Melspectrogram
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        return y, sr, mfccs, mel_spec_db, chroma, contrast
        
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {e}")
        return None, None, None, None, None, None

def load_and_preprocess_data(data_path, sr=44100, duration=2.0, n_mfcc=20):
    """
    Load and preprocess all audio files in the dataset
    """
    X_mel = []
    y = []
    file_paths = []
    classes = ['Healthy', 'Noise', 'Unhealthy']
    class_counts = {}
    
    for i, category in enumerate(classes):
        path = os.path.join(data_path, category)
        print(f"Loading {category} samples...")
        count = 0
        
        for filename in os.listdir(path):
            if not filename.lower().endswith('.wav'):
                continue
                
            file_path = os.path.join(path, filename)
            _, _, _, mel_spec_db, _, _ = extract_features(file_path, sr=sr, duration=duration, n_mfcc=n_mfcc)
            
            if mel_spec_db is not None:
                X_mel.append(mel_spec_db)
                y.append(i)
                file_paths.append(file_path)
                count += 1
        
        class_counts[category] = count
        print(f"  Loaded {count} samples for {category}")
    
    X_mel = np.array(X_mel)
    y = np.array(y)
    
    # Reshape mel spectrograms for CNN input
    X_mel = X_mel.reshape(X_mel.shape[0], X_mel.shape[1], X_mel.shape[2], 1)
    
    return X_mel, y, file_paths, class_counts